# **1. Business Understanding**

## **1.1 Problem Statement**
Individuals and small businesses often struggle to track their **income**, **expenses**, **savings**, and **budgets** in a structured and consistent way. Without a proper financial management system, it becomes difficult to understand where money is going, prevent overspending, or maintain healthy saving habits.

A **Financial Management System (FMS)** helps users organize financial data by storing **transactions**, **budgets**, **accounts**, **categories**, and **savings goals**.  
This project focuses on designing and querying a simple relational database that supports essential financial management features.

## **1.2 Objectives**
The goal of this SQL project is to design a functional **Financial Management System database** and write SQL queries that enable users to:

- Track **income** and **expenses**  
- View **account balances**  
- Categorize transactions (e.g., groceries, rent, salary)  
- Set and monitor **budgets**  
- Track **savings goals**  
- Analyse spending patterns over time  

## **1.3 Key Questions to Answer Using SQL**
The analysis will answer financial questions such as:

- How much did the user spend this month?  
- What are the top categories where money is going?  
- What is the **total income vs total expense** for a given period?  
- Which months had the highest savings?  
- How close is the user to meeting their savings goals?  

## **1.4 Project Scope**
This SQL project will involve:

- Designing the database schema using **ERD principles**  
- Creating database tables in SQL  
- Inserting sample data  
- Running analytical SQL queries  
- Creating **views** for simplified reporting  
- Documenting insights and conclusions  



# **2. Data Understanding**

To build the Financial Management System (FMS), we define a relational database structure that stores essential financial information.  
The system revolves around five main entities: **Users**, **Accounts**, **Categories**, **Transactions**, **Budgets**, and **SavingsGoals**.

Below is a clear description of each table and the role it plays in the system.

## **2.1 Users Table**
Stores information about each user of the financial system.

**Fields:**
- **user_id** (Primary Key)  
- **name**  
- **email**

## **2.2 Accounts Table**
Stores different financial accounts such as bank accounts, mobile money, or cash wallets.

**Fields:**
- **account_id** (Primary Key)  
- **user_id** (Foreign Key → Users.user_id)  
- **account_name**  
- **account_type** (e.g., checking, savings, mobile money, cash)  
- **balance**

## **2.3 Categories Table**
Defines categories for organizing transactions.

**Fields:**
- **category_id** (Primary Key)  
- **category_name** (e.g., groceries, rent, utilities, salary)  
- **category_type** (Income or Expense)

## **2.4 Transactions Table**
Stores detailed financial activities (income and expenses).

**Fields:**
- **transaction_id** (Primary Key)  
- **user_id** (Foreign Key → Users.user_id)  
- **account_id** (Foreign Key → Accounts.account_id)  
- **category_id** (Foreign Key → Categories.category_id)  
- **amount**  
- **transaction_date**  
- **description**

## **2.5 Budgets Table**
Stores monthly budget limits for each spending category.

**Fields:**
- **budget_id** (Primary Key)  
- **user_id** (Foreign Key → Users.user_id)  
- **category_id** (Foreign Key → Categories.category_id)  
- **month**  
- **year**  
- **budget_amount**

## **2.6 SavingsGoals Table**
Tracks user savings goals and progress.

**Fields:**
- **goal_id** (Primary Key)  
- **user_id** (Foreign Key → Users.user_id)  
- **goal_name**  
- **target_amount**  
- **current_amount**  
- **deadline**

## **2.7 Summary**
This data structure ensures the FMS can:

- Store financial activity  
- Support categorization  
- Track budgets  
- Measure savings progress  
- Analyse trends over time  

# **3. Data Preparation**

This step involves creating the full SQL schema for the Financial Management System (FMS) using Python and SQLite.

The following structural components were defined:

- A connection to a SQLite database  
- All required tables for the system  
- Primary keys and foreign keys  
- A database layout capable of supporting income tracking, expense tracking, budgets, and savings goals  

The code block below contains the SQL schema creation.


In [1]:
import sqlite3

# Connect to database (creates a new file if it does not exist)
conn = sqlite3.connect("financial_management_system.db")
cur = conn.cursor()

# Enable foreign keys in SQLite
cur.execute("PRAGMA foreign_keys = ON;")

# 1. Users Table
cur.execute("""
CREATE TABLE IF NOT EXISTS Users (
    user_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL
);
""")

# 2. Accounts Table
cur.execute("""
CREATE TABLE IF NOT EXISTS Accounts (
    account_id INTEGER PRIMARY KEY,
    user_id INTEGER NOT NULL,
    account_name TEXT NOT NULL,
    account_type TEXT NOT NULL,
    balance REAL DEFAULT 0,
    FOREIGN KEY (user_id) REFERENCES Users(user_id)
);
""")

# 3. Categories Table
cur.execute("""
CREATE TABLE IF NOT EXISTS Categories (
    category_id INTEGER PRIMARY KEY,
    category_name TEXT NOT NULL,
    category_type TEXT CHECK (category_type IN ('Income', 'Expense')) NOT NULL
);
""")

# 4. Transactions Table
cur.execute("""
CREATE TABLE IF NOT EXISTS Transactions (
    transaction_id INTEGER PRIMARY KEY,
    user_id INTEGER NOT NULL,
    account_id INTEGER NOT NULL,
    category_id INTEGER NOT NULL,
    amount REAL NOT NULL,
    transaction_date TEXT NOT NULL,
    description TEXT,
    FOREIGN KEY (user_id) REFERENCES Users(user_id),
    FOREIGN KEY (account_id) REFERENCES Accounts(account_id),
    FOREIGN KEY (category_id) REFERENCES Categories(category_id)
);
""")

# 5. Budgets Table
cur.execute("""
CREATE TABLE IF NOT EXISTS Budgets (
    budget_id INTEGER PRIMARY KEY,
    user_id INTEGER NOT NULL,
    category_id INTEGER NOT NULL,
    month INTEGER NOT NULL,
    year INTEGER NOT NULL,
    budget_amount REAL NOT NULL,
    FOREIGN KEY (user_id) REFERENCES Users(user_id),
    FOREIGN KEY (category_id) REFERENCES Categories(category_id)
);
""")

# 6. SavingsGoals Table
cur.execute("""
CREATE TABLE IF NOT EXISTS SavingsGoals (
    goal_id INTEGER PRIMARY KEY,
    user_id INTEGER NOT NULL,
    goal_name TEXT NOT NULL,
    target_amount REAL NOT NULL,
    current_amount REAL DEFAULT 0,
    deadline TEXT,
    FOREIGN KEY (user_id) REFERENCES Users(user_id)
);
""")

conn.commit()
conn.close()

print("Database schema created successfully.")


Database schema created successfully.


# **4. Exploratory Data Analysis (EDA)**

This step focuses on inserting sample data into the database to support exploration and analysis.  
Sample records were added to each table, allowing the structure of the Financial Management System (FMS) to be tested and reviewed.

The inserted data includes:

- Users  
- Financial accounts  
- Income and expense categories  
- Transactions  
- Monthly budgets  
- Savings goals  

The following code block loads the initial dataset into the database.


In [2]:
import sqlite3

conn = sqlite3.connect("financial_management_system.db")
cur = conn.cursor()

# Insert sample Users
cur.executemany("""
INSERT INTO Users (name, email)
VALUES (?, ?);
""", [
    ("Alice Johnson", "alice@example.com"),
    ("Michael Brown", "michael@example.com")
])

# Insert sample Accounts
cur.executemany("""
INSERT INTO Accounts (user_id, account_name, account_type, balance)
VALUES (?, ?, ?, ?);
""", [
    (1, "Equity Bank Account", "Checking", 15000),
    (1, "M-Pesa Wallet", "Mobile Money", 2500),
    (2, "KCB Savings Account", "Savings", 80000)
])

# Insert sample Categories
cur.executemany("""
INSERT INTO Categories (category_name, category_type)
VALUES (?, ?);
""", [
    ("Salary", "Income"),
    ("Groceries", "Expense"),
    ("Rent", "Expense"),
    ("Utilities", "Expense"),
    ("Freelance", "Income")
])

# Insert sample Transactions
cur.executemany("""
INSERT INTO Transactions (user_id, account_id, category_id, amount, transaction_date, description)
VALUES (?, ?, ?, ?, ?, ?);
""", [
    (1, 1, 1, 60000, "2025-01-01", "Monthly salary"),
    (1, 2, 2, -3500, "2025-01-03", "Supermarket shopping"),
    (1, 1, 3, -15000, "2025-01-05", "Monthly rent"),
    (2, 3, 5, 20000, "2025-01-02", "Freelance project"),
    (2, 3, 4, -4500, "2025-01-04", "Electricity bill")
])

# Insert sample Budgets
cur.executemany("""
INSERT INTO Budgets (user_id, category_id, month, year, budget_amount)
VALUES (?, ?, ?, ?, ?);
""", [
    (1, 2, 1, 2025, 12000),
    (1, 3, 1, 2025, 15000),
    (2, 4, 1, 2025, 7000)
])

# Insert sample Savings Goals
cur.executemany("""
INSERT INTO SavingsGoals (user_id, goal_name, target_amount, current_amount, deadline)
VALUES (?, ?, ?, ?, ?);
""", [
    (1, "Emergency Fund", 50000, 10000, "2025-12-31"),
    (2, "New Laptop", 120000, 40000, "2025-06-30")
])

conn.commit()
conn.close()

print("Sample data inserted successfully.")


Sample data inserted successfully.


# **5. Feature Engineering**

This step focuses on creating additional structures that enhance analysis within the Financial Management System (FMS).  
Instead of modifying raw data, new SQL views and computed transformations were added to simplify reporting and analytical queries.

The following engineered features were created:

- A view that summarizes total income and expenses  
- A view that computes spending per category  
- A view that calculates monthly financial summaries  
- A view that tracks progress toward savings goals  

These transformations make it easier to answer financial questions without repeatedly writing complex SQL joins.


In [3]:
import sqlite3

conn = sqlite3.connect("financial_management_system.db")
cur = conn.cursor()

# View 1: Income vs Expense Summary
cur.execute("""
CREATE VIEW IF NOT EXISTS IncomeExpenseSummary AS
SELECT 
    user_id,
    SUM(CASE WHEN category_type = 'Income' THEN amount ELSE 0 END) AS total_income,
    SUM(CASE WHEN category_type = 'Expense' THEN -amount ELSE 0 END) AS total_expense
FROM Transactions
JOIN Categories USING(category_id)
GROUP BY user_id;
""")

# View 2: Spending by Category
cur.execute("""
CREATE VIEW IF NOT EXISTS CategorySpending AS
SELECT 
    user_id,
    category_name,
    SUM(-amount) AS total_spent
FROM Transactions
JOIN Categories USING(category_id)
WHERE category_type = 'Expense'
GROUP BY user_id, category_name;
""")

# View 3: Monthly Summary
cur.execute("""
CREATE VIEW IF NOT EXISTS MonthlySummary AS
SELECT 
    user_id,
    SUBSTR(transaction_date, 1, 7) AS year_month,
    SUM(CASE WHEN category_type = 'Income' THEN amount ELSE 0 END) AS income,
    SUM(CASE WHEN category_type = 'Expense' THEN -amount ELSE 0 END) AS expenses,
    SUM(CASE WHEN category_type = 'Income' THEN amount ELSE 0 END)
      - SUM(CASE WHEN category_type = 'Expense' THEN -amount ELSE 0 END) AS savings
FROM Transactions
JOIN Categories USING(category_id)
GROUP BY user_id, year_month;
""")

# View 4: Savings Goal Progress
cur.execute("""
CREATE VIEW IF NOT EXISTS SavingsProgress AS
SELECT 
    user_id,
    goal_name,
    target_amount,
    current_amount,
    ROUND((current_amount * 100.0) / target_amount, 2) AS progress_percentage
FROM SavingsGoals;
""")

conn.commit()
conn.close()

print("Feature engineering views created successfully.")


Feature engineering views created successfully.


# **6. Modeling**

This step focuses on writing SQL queries that extract meaningful insights from the Financial Management System (FMS).  
The queries are designed to answer key financial questions such as total monthly spending, category-level expenses, income versus expense comparisons, and savings progress.

The following SQL statements demonstrate how the engineered views and raw tables can be used to perform financial analysis.


In [4]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("financial_management_system.db")

# 1. Total income and expense summary per user
query1 = """
SELECT *
FROM IncomeExpenseSummary;
"""
df1 = pd.read_sql(query1, conn)
print("Income vs Expense Summary:")
display(df1)

# 2. Top spending categories per user
query2 = """
SELECT *
FROM CategorySpending
ORDER BY total_spent DESC;
"""
df2 = pd.read_sql(query2, conn)
print("Category Spending:")
display(df2)

# 3. Monthly income, expense, and savings summary
query3 = """
SELECT *
FROM MonthlySummary
ORDER BY year_month;
"""
df3 = pd.read_sql(query3, conn)
print("Monthly Summary:")
display(df3)

# 4. Savings goal progress
query4 = """
SELECT *
FROM SavingsProgress;
"""
df4 = pd.read_sql(query4, conn)
print("Savings Goal Progress:")
display(df4)

# 5. Total spending for a specific month (example: January 2025)
query5 = """
SELECT 
    user_id,
    SUM(-amount) AS january_spending
FROM Transactions
JOIN Categories USING(category_id)
WHERE category_type = 'Expense'
  AND SUBSTR(transaction_date, 1, 7) = '2025-01'
GROUP BY user_id;
"""
df5 = pd.read_sql(query5, conn)
print("January 2025 Spending:")
display(df5)

# 6. Income vs Expense for a chosen period (example: 2025-01)
query6 = """
SELECT 
    user_id,
    SUM(CASE WHEN category_type = 'Income' THEN amount ELSE 0 END) AS income,
    SUM(CASE WHEN category_type = 'Expense' THEN -amount ELSE 0 END) AS expense
FROM Transactions
JOIN Categories USING(category_id)
WHERE SUBSTR(transaction_date, 1, 7) = '2025-01'
GROUP BY user_id;
"""
df6 = pd.read_sql(query6, conn)
print("Income vs Expense (January 2025):")
display(df6)

conn.close()


Income vs Expense Summary:


,user_id,total_income,total_expense
0,1,60000.0,18500.0
1,2,20000.0,4500.0


Category Spending:


,user_id,category_name,total_spent
0,1,Rent,15000.0
1,2,Utilities,4500.0
2,1,Groceries,3500.0


Monthly Summary:


...

,user_id,goal_name,target_amount,current_amount,progress_percentage
0,1,Emergency Fund,50000.0,10000.0,20.00
1,2,New Laptop,120000.0,40000.0,33.33


January 2025 Spending:


,user_id,january_spending
0,1,18500.0
1,2,4500.0


Income vs Expense (January 2025):


,user_id,income,expense
0,1,60000.0,18500.0
1,2,20000.0,4500.0


# **7. Evaluation**

This step evaluates the performance of the Financial Management System (FMS) using SQL-based checks.  
The goal is to confirm that the database structure and analytical queries produce accurate and reliable financial insights.

The evaluation covers:

- Accuracy of computed financial summaries  
- Ability to track income and expenses  
- Clarity of category-based spending breakdowns  
- Usefulness of monthly financial insights  
- Effectiveness of savings goal monitoring  

The following code block performs validation checks using SQL queries.


In [5]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("financial_management_system.db")

# 1. Check accuracy of income and expense totals
print("1. Income vs Expense Summary Validation:")
q1 = """
SELECT 
    user_id,
    SUM(CASE WHEN category_type = 'Income' THEN amount ELSE 0 END) AS calculated_income,
    SUM(CASE WHEN category_type = 'Expense' THEN -amount ELSE 0 END) AS calculated_expense
FROM Transactions
JOIN Categories USING(category_id)
GROUP BY user_id;
"""
display(pd.read_sql(q1, conn))

print("Using the engineered view:")
display(pd.read_sql("SELECT * FROM IncomeExpenseSummary;", conn))


# 2. Validate category-level spending
print("\n2. Category Spending Validation:")
q2 = """
SELECT 
    user_id,
    category_name,
    SUM(-amount) AS calculated_spending
FROM Transactions
JOIN Categories USING(category_id)
WHERE category_type = 'Expense'
GROUP BY user_id, category_name
ORDER BY calculated_spending DESC;
"""
display(pd.read_sql(q2, conn))

print("Using the engineered view:")
display(pd.read_sql("SELECT * FROM CategorySpending;", conn))


# 3. Validate monthly summaries
print("\n3. Monthly Summary Validation:")
q3 = """
SELECT 
    user_id,
    SUBSTR(transaction_date, 1, 7) AS year_month,
    SUM(CASE WHEN category_type = 'Income' THEN amount ELSE 0 END) AS income,
    SUM(CASE WHEN category_type = 'Expense' THEN -amount ELSE 0 END) AS expenses,
    SUM(CASE WHEN category_type = 'Income' THEN amount ELSE 0 END)
      - SUM(CASE WHEN category_type = 'Expense' THEN -amount ELSE 0 END) AS savings
FROM Transactions
JOIN Categories USING(category_id)
GROUP BY user_id, year_month;
"""
display(pd.read_sql(q3, conn))

print("Using the engineered view:")
display(pd.read_sql("SELECT * FROM MonthlySummary;", conn))


# 4. Validate savings goal calculations
print("\n4. Savings Goal Validation:")
q4 = """
SELECT 
    goal_name,
    target_amount,
    current_amount,
    ROUND((current_amount * 100.0) / target_amount, 2) AS calculated_percentage
FROM SavingsGoals;
"""
display(pd.read_sql(q4, conn))

print("Using the engineered view:")
display(pd.read_sql("SELECT * FROM SavingsProgress;", conn))

conn.close()


1. Income vs Expense Summary Validation:


,user_id,calculated_income,calculated_expense
0,1,60000.0,18500.0
1,2,20000.0,4500.0


Using the engineered view:


,user_id,total_income,total_expense
0,1,60000.0,18500.0
1,2,20000.0,4500.0



2. Category Spending Validation:


...

,user_id,year_month,income,expenses,savings
0,1,2025-01,60000.0,18500.0,41500.0
1,2,2025-01,20000.0,4500.0,15500.0



4. Savings Goal Validation:


,goal_name,target_amount,current_amount,calculated_percentage
0,Emergency Fund,50000.0,10000.0,20.00
1,New Laptop,120000.0,40000.0,33.33


Using the engineered view:


,user_id,goal_name,target_amount,current_amount,progress_percentage
0,1,Emergency Fund,50000.0,10000.0,20.00
1,2,New Laptop,120000.0,40000.0,33.33
